In [136]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import sys

In [137]:
def search(driver, date):
    """
    Date in mm/dd/yyyy
    """
    try:
        driver.get("https://merolagani.com/Floorsheet.aspx")
        try:
            alert = driver.switch_to.alert
            print(f'Alert detected {alert.text}')
            alert.dismis()
        except Exception as e:
            pass
        date_element_path = '/html/body/form/div[4]/div[4]/div/div/div[1]/div[4]/input'
        search_element_path2 = '/html/body/form/div[4]/div[4]/div/div/div[2]/a[1]'
        date_input = driver.find_element(By.XPATH, date_element_path)
        search_btn = driver.find_element(By.XPATH, search_element_path2)
        # date_input = driver.find_element_by_xpath()
        # search_btn = driver.find_element_by_xpath()
        date_input.send_keys('02/13/2025')
        print(search_btn)
        search_btn.click()
    except Exception as e:
        print(e)
    # if driver.find_elements(By.XPATH, "//*[contains(text(), 'Could not find floorsheet matching the search criteria')]"):
    #     print("No data found for the given search.")
    #     print("Aborting script ......")
    #     sys.exit()

In [138]:
date

'02/15/2025'

In [139]:
def get_page_table(driver, table_class):
    soup = BeautifulSoup(driver.page_source,'html')
    table = soup.find("table", {"class":table_class})
    tab_data = [[cell.text.replace('\r', '').replace('\n', '') for cell in row.find_all(["th","td"])]
                        for row in table.find_all("tr")]
    df = pd.DataFrame(tab_data)
    return df

In [140]:
def scrape_data(driver, date):
    start_time = datetime.now()
    search(driver,'02/13/2025')
    df = pd.DataFrame()
    while True:
        page_table_df = get_page_table(driver, table_class="table table-bordered table-striped table-hover sortable")
        print(page_table_df)
        df = df.append(page_table_df, ignore_index = True)
        try:
            next_btn = driver.find_element_by_link_text('Next')
            driver.execute_script("arguments[0].click();", next_btn)
        except NoSuchElementException:
            break
    print(f"Time taken to scrape: {datetime.now() - start_time}")    
    return df

In [141]:
def clean_df(df):
    new_df = df.drop_duplicates(keep='first') # Dropping Duplicates
    new_header = new_df.iloc[0] # grabing the first row for the header
    new_df = new_df[1:] # taking the data lower than the header row
    new_df.columns = new_header # setting the header row as the df header
    new_df.drop(["#"], axis=1, inplace=True)
    new_df["Rate"] = new_df["Rate"].apply(lambda x:float(x.replace(",", ""))) # Convert Rate to Float
    new_df["Amount"] = new_df["Amount"].apply(lambda x:float(x.replace(",", ""))) # Convert Amount to Float
    return new_df

In [142]:
options = Options()
options.headless = True
options.add_argument('--disable-notifications')
driver = webdriver.Chrome(options=options) # Start Browser

date = datetime.today().strftime('%m/%d/%Y') # Get today's date
search(driver, '02/13/2025') # Search the webpage
df = scrape_data(driver, date) # Scraping
final_df = clean_df(df) # Cleaning

<selenium.webdriver.remote.webelement.WebElement (session="be1a4f5d6dbb4c35c96bc9985512fdb7", element="f.1AAED206929C755148A248E0C7B90772.d.7E01DD821CB5A16B29975620F1F31D7C.e.93")>
Alert Text: Do you want to receive notifications?
Message: unexpected alert open: {Alert text : Do you want to receive notifications?}
  (Session info: chrome=133.0.6943.98)
Stacktrace:
#0 0x60eac4649bba <unknown>
#1 0x60eac40e7790 <unknown>
#2 0x60eac41857a0 <unknown>
#3 0x60eac415e823 <unknown>
#4 0x60eac412aa88 <unknown>
#5 0x60eac412bbf1 <unknown>
#6 0x60eac461315b <unknown>
#7 0x60eac46170e2 <unknown>
#8 0x60eac460001c <unknown>
#9 0x60eac4617cd4 <unknown>
#10 0x60eac45e448f <unknown>
#11 0x60eac46384f8 <unknown>
#12 0x60eac46386c9 <unknown>
#13 0x60eac4648a36 <unknown>
#14 0x7934d969caa4 <unknown>
#15 0x7934d9729c3c <unknown>

       0                 1       2      3       4         5         6  \
0      #     Transact. No.  Symbol  Buyer  Seller  Quantity      Rate   
1      1  2025021305016233     B

AttributeError: 'DataFrame' object has no attribute 'append'

In [7]:
final_df.head()

NameError: name 'final_df' is not defined

In [ ]:
file_name = date.replace("/", "_")
final_df.to_csv(f"data/{file_name}.csv", index=False) # Save file